<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env: deepcell]</code> for this notebook. </div>

# 1. Inference

In [1]:
import os
import sys

import numpy as np
import tensorflow as tf

from deepcell.applications import NuclearSegmentation, CellTracking
from deepcell.datasets import DynamicNuclearNetTracking

sys.path.append('..')
import utils

In [3]:
data_dir = 'data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

for d in [data_dir, gt_seg_dir, pred_seg_dir]:
    if not os.path.exists(d):
        os.makedirs(d)

In [9]:
# Load test data
dnn = DynamicNuclearNetTracking(version='1.1')
X, y, lineages = dnn.load_data(split='test')
data = {
    'X': X,
    'y': y,
    'lineages': lineages
}

INFO:root:Checking for cached data
INFO:root:Checking DynamicNuclearNet-tracking-v1_1.zip against provided file_hash...
INFO:root:DynamicNuclearNet-tracking-v1_1.zip with hash 4090ccd3b76c27f001f798c8187c5f3e already available.
INFO:root:Extracting /home/msschwar/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip
INFO:root:Successfully extracted /home/msschwar/.deepcell/datasets/DynamicNuclearNet-tracking-v1_1.zip into /home/msschwar/.deepcell/datasets


In [5]:
# Load segmentation and tracking applications
app_seg = NuclearSegmentation.from_version(version='1.1')
app_trk = CellTracking.from_version(version='1.1')

INFO:root:Checking for cached data
INFO:root:Making request to server
INFO:root:Downloading models/NuclearSegmentation-8.tar.gz with size 429.5 MB to /home/msschwar/.deepcell/models
429MB [00:19, 23.6MB/s]                            
INFO:root:🎉 Successfully downloaded file to /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearSegmentation-8.tar.gz into /home/msschwar/.deepcell/models
2024-07-01 11:53:24.460674: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 11:53:25.424471: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Cre

INFO:root:Checking for cached data
INFO:root:Making request to server
INFO:root:Downloading models/NuclearTrackingNE-8.tar.gz with size 629.5 KB to /home/msschwar/.deepcell/models
630kB [00:00, 4.70MB/s]                           
INFO:root:🎉 Successfully downloaded file to /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearTrackingNE-8.tar.gz into /home/msschwar/.deepcell/models


INFO:root:Checking for cached data
INFO:root:Making request to server
INFO:root:Downloading models/NuclearTrackingInf-8.tar.gz with size 548.6 KB to /home/msschwar/.deepcell/models
549kB [00:00, 4.19MB/s]                           
INFO:root:🎉 Successfully downloaded file to /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz
INFO:root:Extracting /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz
INFO:root:Successfully extracted /home/msschwar/.deepcell/models/NuclearTrackingInf-8.tar.gz into /home/msschwar/.deepcell/models


In [10]:
for batch_no in range(len(data['lineages'])):
    print(batch_no)

    # Pull out relevant data for this batch
    X = data['X'][batch_no]
    y = data['y'][batch_no]
    lineage = data['lineages'][batch_no]

    # Correct discontiguous tracks, which are not allowed by CTC
    y, lineage = utils.convert_to_contiguous(y, lineage)

    # Determine position of zero padding for removal
    slc = utils.find_zero_padding(X)
    X = X[slc]
    y = y[slc]

    # Determine which frames are zero padding
    frames = np.sum(y, axis=(1,2)) # True if image not blank
    good_frames = np.where(frames)[0]
    X = X[:len(good_frames)]
    y = y[:len(good_frames)]

    # Generate tracks on GT segmentations
    track_gt = app_trk.track(X, y)
    track_gt['y_tracked'], track_gt['tracks'] = utils.convert_to_contiguous(track_gt['y_tracked'], track_gt['tracks'])
    utils.save_ctc_res(gt_seg_dir, batch_no + 1, track_gt['y_tracked'][..., 0], track_gt['tracks'])
    utils.save_ctc_gt(gt_seg_dir, batch_no + 1, y[..., 0], lineage)

    # Generate tracks on predicted segmentations
    y_pred = app_seg.predict(X)
    track_pred = app_trk.track(X, y_pred)
    track_pred['y_tracked'], track_pred['tracks'] = utils.convert_to_contiguous(track_pred['y_tracked'], track_pred['tracks'])
    utils.save_ctc_res(pred_seg_dir, batch_no + 1, track_pred['y_tracked'][..., 0], track_pred['tracks'])
    utils.save_ctc_gt(pred_seg_dir, batch_no + 1, y[..., 0], lineage)

0


2024-07-01 11:56:59.900345: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-07-01 11:56:59.915177: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8800
2024-07-01 11:57:00.055043: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-01 11:57:00.057358: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-01 11:57:00.057385: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-07-01 11:57:00.059780: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-01 11:57:00.059859: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to

1


INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Tracked frame 1 in 0.09035235404735431 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Tracked frame 2 in 0.06955858098808676 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Tracked frame 3 in 0.07533263601362705 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Tracked frame 4 in 0.0739827870274894 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Tracked frame 5 in 0.0709641199791804 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Tracked frame 6 in 0.07815538498107344 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Tracked frame 7 in 0.07044252101331949 s.
INFO:CellTracker:Tracking frame 8
INFO:CellTracker:Tracked frame 8 in 0.0712150099570863 s.
INFO:CellTracker:Tracking frame 9
INFO:CellTracker:Tracked frame 9 in 0.07060693996027112 s.
INFO:CellTracker:Tracking frame 10
INFO:CellTracker:Tracked frame 10 in 0.07331035798415542 s.
INFO:CellTracker:Tracking frame 11
INFO:CellTracker:Tracked frame 11 in

2


...

INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Tracked frame 1 in 0.09448268701089546 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 79.
INFO:CellTracker:Tracked frame 2 in 0.08418465300928801 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Tracked frame 3 in 0.08472914301091805 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 80.
INFO:CellTracker:Tracked frame 4 in 0.08510455203941092 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Created new track for cell 81.
INFO:CellTracker:Detected division! Cell 81 is daughter of cell 43.
INFO:CellTracker:Tracked frame 5 in 0.0828129660221748 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 83.
INFO:CellTracker:Created new track for cell 84.
INFO:CellTracker:Detected division! Cell 84 is daughter of cell 38.
INFO:CellTracker:Created new track for cell 85.
INFO:CellTracker:Detected division! Cell 85 is daughter of cell 38.
INFO:CellT

10


INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Tracked frame 1 in 0.10712135600624606 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Tracked frame 2 in 0.09296611999161541 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 124.
INFO:CellTracker:Tracked frame 3 in 0.09518657700391486 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 125.
INFO:CellTracker:Tracked frame 4 in 0.10025607701390982 s.
INFO:CellTracker:Tracking frame 5
INFO:CellTracker:Tracked frame 5 in 0.09038408502237871 s.
INFO:CellTracker:Tracking frame 6
INFO:CellTracker:Created new track for cell 126.
INFO:CellTracker:Detected division! Cell 126 is daughter of cell 117.
INFO:CellTracker:Created new track for cell 127.
INFO:CellTracker:Detected division! Cell 127 is daughter of cell 117.
INFO:CellTracker:Tracked frame 6 in 0.09965355898020789 s.
INFO:CellTracker:Tracking frame 7
INFO:CellTracker:Tracked frame 7 in 0.09210588200949132 s.
INFO:CellTrac

11


INFO:CellTracker:Tracking frame 1
INFO:CellTracker:Created new track for cell 180.
INFO:CellTracker:Created new track for cell 181.
INFO:CellTracker:Created new track for cell 182.
INFO:CellTracker:Created new track for cell 183.
INFO:CellTracker:Created new track for cell 184.
INFO:CellTracker:Created new track for cell 185.
INFO:CellTracker:Tracked frame 1 in 0.147874966962263 s.
INFO:CellTracker:Tracking frame 2
INFO:CellTracker:Created new track for cell 186.
INFO:CellTracker:Created new track for cell 187.
INFO:CellTracker:Created new track for cell 188.
INFO:CellTracker:Created new track for cell 189.
INFO:CellTracker:Tracked frame 2 in 0.12075253599323332 s.
INFO:CellTracker:Tracking frame 3
INFO:CellTracker:Created new track for cell 190.
INFO:CellTracker:Tracked frame 3 in 0.10889445804059505 s.
INFO:CellTracker:Tracking frame 4
INFO:CellTracker:Created new track for cell 191.
INFO:CellTracker:Created new track for cell 192.
INFO:CellTracker:Detected division! Cell 192 is daug

# 2. Evaluation

In [11]:
import glob
import os
import re
import subprocess

import numpy as np
import pandas as pd

from deepcell_tracking.metrics import TrackingMetrics

In [13]:
data_dir = 'data'
gt_seg_dir = os.path.join(data_dir, 'SEG_GT')
pred_seg_dir = os.path.join(data_dir, 'SEG_PRED')

pattern = re.compile('\d{3}_GT')
data_ids = [f.split('_')[0] for f in os.listdir(gt_seg_dir) if pattern.fullmatch(f)]

node_match_threshold = 0.6

ctc_software = '../CTC_Evaluation_Software'
operating_system = 'Linux' # or 'Mac' or 'Win'
num_digits = '3'

In [14]:
benchmarks = []

for results_dir, s in zip([gt_seg_dir, pred_seg_dir], ['GT', 'Caliban']):
    for data_id in data_ids:
        results = {
            'model': f'Caliban - {s}',
            'data_id': os.path.splitext(data_id)[0]
        }
        gt_dir = os.path.join(results_dir, f'{data_id}_GT/TRA')
        res_dir = os.path.join(results_dir, f'{data_id}_RES')

        # Deepcell benchmarking
        m = TrackingMetrics.from_isbi_dirs(gt_dir, res_dir, threshold=node_match_threshold)
        results.update(m.stats)

        # CTC metrics
        for metric, path in [('DET', 'DETMeasure'), ('SEG', 'SEGMeasure'), ('TRA', 'TRAMeasure')]:
            p = subprocess.run([os.path.join(ctc_software, operating_system, path), results_dir, data_id, num_digits],
                               stdout=subprocess.PIPE)
            outstring = p.stdout

            try:
                val = float(outstring.decode('utf-8').split()[-1])
                results[metric] = val
            except:
                print('Benchmarking failure', path, results_dir, data_id)
                print(outstring.decode('utf-8'))

        benchmarks.append(results)

df = pd.DataFrame(benchmarks)
df.to_csv('benchmarks.csv')

missed node 135_25 division completely
missed node 144_28 division completely
missed node 148_59 division completely
missed node 21_34 division completely
missed node 26_25 division completely
missed node 1_29 division completely
missed node 57_10 division completely
missed node 60_0 division completely
5_53 out degree = 1, daughters mismatch.
12_31 out degree = 2, daughters mismatch, gt and res degree equal.
18_16 out degree = 2, daughters mismatch, gt and res degree equal.
35_29 out degree = 2, daughters mismatch, gt and res degree equal.
missed node 30_37 division completely
missed node 67_40 division completely
missed node 86_7 division completely
missed node 104_34 division completely
missed node 112_6 division completely
missed node 121_42 division completely
126_61 out degree = 1, daughters mismatch.
missed node 127_51 division completely
missed node 135_25 division completely
missed node 144_28 division completely
missed node 148_59 division completely
missed node 163_66 divisi

In [15]:
df

,model,data_id,correct_division,mismatch_division,false_positive_division,false_negative_division,total_divisions,aa_tp,aa_total,te_tp,te_total,DET,SEG,TRA
0,Caliban - GT,001,54,0,10,3,57,14532,14806,14900,15174,1.000000,1.000000,0.999601
1,Caliban - GT,002,1,0,0,0,1,1075,1075,1109,1109,1.000000,1.000000,1.000000
2,Caliban - GT,003,1,0,0,2,3,992,997,1025,1030,1.000000,1.000000,0.999576
3,Caliban - GT,004,1,0,0,0,1,517,521,536,540,1.000000,1.000000,0.999757
4,Caliban - GT,005,7,0,0,1,8,2049,2055,2116,2122,1.000000,1.000000,0.999836
5,Caliban - GT,006,2,0,0,0,2,690,690,712,712,1.000000,1.000000,1.000000
6,Caliban - GT,007,18,0,2,2,20,4914,4945,5025,5056,1.000000,1.000000,0.999905
7,Caliban - GT,008,0,0,0,0,0,356,383,370,397,1.000000,1.000000,0.995929
8,Caliban - GT,009,1,0,0,0,1,180,199,187,206,1.000000,1.000000,0.998094
9,Caliban - GT,010,15,0,1,0,15,3796,3833,3938,3975,1.000000,1.000000,0.999737
